In [2]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
from bq_helper import BigQueryHelper
%load_ext google.cloud.bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="My Project-bbdce7b1712b.json"

bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")

query="""
SELECT * FROM `bigquery-public-data.san_francisco.sfpd_incidents` 
"""

df = bq_assistant.query_to_pandas(query)


In [6]:
df[df['unique_key']==80460270]

,unique_key,category,descript,dayofweek,pddistrict,resolution,address,longitude,latitude,location,pdid,timestamp
9,80460270,DISORDERLY CONDUCT,COMMITTING PUBLIC NUISANCE,Friday,PARK,"ARREST, BOOKED",HAIGHT ST / BUENAVISTAWEST AV,-122.443312,37.77047,"(37.7704697548539, -122.443311949602)",8046027019010,2008-05-02 14:31:00+00:00
280,80460270,DRUNKENNESS,UNDER INFLUENCE OF ALCOHOL IN A PUBLIC PLACE,Friday,PARK,"ARREST, BOOKED",HAIGHT ST / BUENAVISTAWEST AV,-122.443312,37.77047,"(37.7704697548539, -122.443311949602)",8046027019090,2008-05-02 14:31:00+00:00
942761,80460270,DRUG/NARCOTIC,POSSESSION OF MARIJUANA,Friday,PARK,"ARREST, BOOKED",HAIGHT ST / BUENAVISTAWEST AV,-122.443312,37.77047,"(37.7704697548539, -122.443311949602)",8046027016010,2008-05-02 14:31:00+00:00


In [7]:
df.describe(include="object")

,category,descript,dayofweek,pddistrict,resolution,address,location
count,2170785,2170785,2170785,2170785,2170785,2170785,2170785
unique,39,915,7,11,17,25123,60768
top,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Friday,SOUTHERN,NONE,800 Block of BRYANT ST,"(37.775420706711, -122.403404791479)"
freq,467657,173079,331007,391669,1356705,64271,55669


# Bar Charts from 2003 to 2013 

## Top 5 Types of Crimes in terms of No of Incidents Reported

In [44]:
df_temp=pd.read_csv('crime rate/results-20181121-154239.csv')
## ref https://stackoverflow.com/questions/17141558/how-to-sort-a-dataframe-in-python-pandas-by-two-or-more-columns
df_temp.sort_values(by=['Year','count_incidents'], ascending=[True,True],inplace=True)

In [64]:
# For visualization
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Spectral6, brewer
from bokeh.transform import factor_cmap


## Reference https://stackoverflow.com/questions/16958499/sort-pandas-dataframe-and-print-highest-n-values
for yr, df_year in df_temp.groupby('Year'):
    ##print(df_year.nlargest(10,'count_incidents'))
#     categ_list=df_year.nlargest(10,'count_incidents')['Category'].tolist()
#     count_incidents_list=df_year.nlargest(10,'count_incidents')['count_incidents'].tolist()
    
    output_file('plot_of_incidents_by_'+str(yr)+'.html')
    
    source = ColumnDataSource(df_year.nlargest(5,'count_incidents'))
    categ_list = source.data['Category'].tolist()
    
    p = figure(x_range=categ_list, plot_width=900, plot_height=800)

    color_map = factor_cmap(field_name='Category', palette=Spectral6, factors=categ_list)

    p.vbar(x='Category', top='count_incidents', source=source, width=0.70, color=color_map)

    p.title.text ='No of Reported Incidents per Category for '+str(yr)
    p.xaxis.axis_label = 'Category Type'
    p.yaxis.axis_label = "No of Incidents in Year: "+str(yr)

    show(p)
    

In [66]:
# grouped = df_temp.groupby('Year')['count_incidents'].max()
# ser_df=pd.DataFrame({'Year':grouped.index, 'count_incidents':grouped.values})
# ser_df

In [65]:
# result = pd.merge(df_temp, ser_df, how='inner', on=['count_incidents', 'Year'])
# result